<CENTER><img src="images/logos.png" style="width:50%"></CENTER>

# Unknown three body decay <a name="c"></a>

**The following analysis aims to find the decay process. We will know which are the final state particles and we want to search the decay.**

In this tutorial we will start from pions and kaons as final state particles, and we aim to find the combination for which the invariant mass distribution looks like a gaussian centered at the value corresponding to the mass of the decaying particle. If we find the correct combination, we discover the decaying particle due to its mass, and the decay process.

The pion, $\pi$, is composed of a $u$ quark and an $\bar{d}$ antiquark and the kaon is composed of a $u$ quark and an $\bar{s}$ antiquark (as we've seen in the Introduction to Particle Physics notebook). In this notebook we want to reconstruct the trajectories of our final state particles, our pions and kaons, in order to find the mass of the decaying meson. We don't know what is the combination yet, the only thing we know is that our final state is constructed of pions and kaons!

We will be dealing with a three-body decay, the total electric charge must be conserved! 

We've learned in the previous notebook about 4-vectors and we know how to manipulate them. You will see now why was so important to learn about 4-vectors and what we can do with them in real life! We will look at our misterious final state particles but firstly, we need to think about how we talk about decays in particle physcis.

**Contents:**
- [Decays](#1.)
- [The invariant mass calculation](#2.)
- [Cuts](#3.)
- [Find the mistery decaying particle!](#4.)

---

## Decays <a name="1."></a>

Let's start with understanding the idea of a decay process!

We want to find the decaying meson, but they don't live long enough to actually see with the detector itself. Instead, we have to **reconstruct** them from their decay products. We don't particularly mind where the the decaying mesons themselves come from. All you need to know here is that each time the LHC smashes two protons together, they produce lots and lots of particles.



In this process, our decaying meson emerges from the collision aftermath in the LHC (we don't care how) can decay directly to pions and kaons. The decay happens at the vertex below, where the two black arrows meet. Here our final state particles labeled as $h$ can be either pions or kaons, and $X$ is our decaying particle to be identified.

<figure>
    <center> <img src="images/3body-decay.png" alt="TEST" style="width:40%" />
   <figcaption>Image 1: The misterious X decay directly from the proton-proton collision. </figcaption> </center>
</figure>

The LHCb detector can measure the momentum and energy of the final state particles coming out of decays, and you can access that information quite simply. We mentioned earlier that each smash makes lots of particles. That's true - in fact, it makes so many that we can't actually store all the records of what happened, even on some of the biggest data storage facilities in the world. 



Next we have to open the data that we want to analyze. As described earlier, the data is stored in a *.root file. We can use a python library called uproot to access the data. Below is an example of how to open a *.root file using uproot

In [ ]:
## data file
import uproot
f = uproot.open("../LHCb_data/3Body/Unknown3body.root")

We can inspect the contents of a file by using the method keys()

In [ ]:
f.keys()

We see that we have an object called 'DecayTree'. We can obtain information about the object in the file and its type by using the method classnames()

In [ ]:
f.classnames()

We see that the object called DecayTree is a TTree type. A TTree is simply columns of data stored in the .root format. Each column of data can represent a different physical quantity of a particle. For instance, its charge, energy, momentum etc.

Now we know what data the file contains, in future we can quickly access that data. We want to access the mini data. This can be done by executing the command below


In [ ]:
events = uproot.open("../LHCb_data/3Body/Unknown3body.root:DecayTree")

Let's look at contents of the TTree. Essentially all the columns in the TTree called DecayTree

In [ ]:
events.keys()

We basically have the three momentum components for our three particles. We can use the .arrays method to access events with just the columns we specify.

We want to run over all the data and reconstruct the X meson mass. To do this we will access events using the arrays method. Let's look at doing this.

First we define a histogram. To do this we can import the python hist library. Once we have done that we can define a histogram. Its name is hist and the x axis is named m (MeV). For instance, here is an example of how you define a histogram. The three initial arguments indicate that this histogram contains 100 bins which fill the gap from 0,150000.


In [ ]:
import hist
from hist import Hist

hist_mass = Hist(hist.axis.Regular(100,0,150000, label = "m (MeV)"))

It is now time to fill our above defined histogram with the masses. To do that, we need to reconstruct our X meson invariant mass. 


[Return to contents](#c)

---

## The invariant mass calculation <a name="2."></a>

One very important quantity we need to define is the invariant mass! It is defined as an invariant quantity which is the same for all observers in all reference frames (that's why we call it *invariant*). To calculate it we use the energy, *E*, and momentum, *p*, both measured in the detector. To derive a proper expression for the invariat mass we ask that in the process both energy and momentum are conserved! 

* Energy conservation

$ E = E_{h1} + E_{h2} + E_{h3} $

* Momentum conservation

$ \vec{p} = \vec{p}_{h1} + \vec{p}_{h2} + \vec{p}_{h3} $

From special relativity we've learned the relation between mass, energy and momentum to be:

$ (pc)^2 + (mc^2)^2 = E^2 $

$ E^2 = p^2 + m^2$

Rearranging it to $m$:

$ m^2 = E^2 - p2 = (E_{h1} + E_{h2} + E_{h3})^2 - || p_{h1} + p_{h2} + p_{h3} ||$

$ m^2 = (E_{h1}+ E_{h2} + E_{h3})^2 - (\vec{p}_{h1} + \vec{p}_{h2} + \vec{p}_{h3})\cdot (\vec{p}_{h1} + \vec{p}_{h2} + \vec{p}_{h3})$

Since the dot product of two orthogonal vectors is zero, we can write more explicitly

$ m^2 = (E_{h1} + E_{h2} + E_{h3})^2 - (p_{h1_x} + p_{h2_x} + p_{h3_x} )^2 - (p_{h1_y} + p_{h2_y} + p_{h3_y})^2 - (p_{h1_z} + p_{h2_z} + p_{h3_z})^2$

<div class="alert alert-info"> Our goal is to calculte the invariant mass of the three chosen particles and we save it to a histogram.</div> 

## Extracting momentum components and calculating the energy distributions


In [ ]:
# Extracting the momentum components and energy of the first muon
p1x = events["p1_PX"].array(library="np")
p1y = events["p1_PY"].array(library="np")
p1z = events["p1_PZ"].array(library="np")

Let's visualise one of these momentum components, for instance let's check the x-component of the one of our final particles! To do that we first define a histogram, and we fill it with the values we extracted in `p1x`.

In [ ]:
import hist
from hist import Hist
import matplotlib.pyplot as plt
import numpy as np

# Defining histogram
hist_p1x = Hist(hist.axis.Regular(100,-50e3,50e3, label = "$p_{1x}$ (MeV)"))

# Fiblling histogram with values of p1x
hist_p1x.fill(p1x)

# Plotting the histogram
hist_p1x.plot()
plt.show()

Great! Momentum is a **vector** quantity, it has x,y and z components. Now let's try to calculate the magnitude of the momentum, $p^{2} = p_{x}^{2} + p_{y}^{2} + p_{z}^{2}$, of the first candidate and plot it into a histogram: 

In [ ]:
# Calulating the magnitude of the momentum:
p1 = np.sqrt( p1x**2 + p1y**2 + p1z**2)

# Defining histogram
hist_p1 = Hist(hist.axis.Regular(100,0e3,50e3, label = "$p_{1}$ (MeV)")) # *** axis label: meson not muon

# Fiblling histogram with values of p1
hist_p1.fill(p1)

# Plotting the histogram
hist_p1.plot()
plt.show()

### Your turn!

We've calculated the magnitude of the momentum for the first final state particle. Repeat that now for the other two!

The first step is to extract the momentum components. Then plot one of the components as an example, for instace the x-component ($p_{2x}$ and $p_{3x}$). Finally, calculate and plot the magnitude of the momentum for the second and third final state particles.

In [ ]:
# Extracting the momentum components and energy of the second and third final state particles
p2x = #COMPLETE
p2y = #COMPLETE
p2z = #COMPLETE

p3x = #COMPLETE
p3y = #COMPLETE
p3z = #COMPLETE

# Defining histogram
hist_p2x = Hist(hist.axis.Regular(100,-50e3,50e3, label = "$p_{2x}$ (MeV)"))
hist_p3x = Hist(hist.axis.Regular(100,-50e3,50e3, label = "$p_{3x}$ (MeV)"))

# Filling histogram with values of p2x and p3x
hist_p2x.fill#COMPLETE
hist_p3x.fill#COMPLETE

# Plotting the histograms
fig, axes = plt.subplots(figsize=(10,5))
plt.subplot(1,2,1)
hist_p2x.plot()
plt.subplot(1,2,2)
hist_p3x.plot()
plt.show()

plt.show()

In [ ]:
# Calulating the magnitude of the momentum:
p2 = #COMPLETE
p3 = #COMPLETE

# Defining histogram
hist_p2 = Hist(hist.axis.Regular(100,0e3,50e3, label = "$p_{2}$ (MeV)"))
hist_p3 = Hist(hist.axis.Regular(100,0e3,50e3, label = "$p_{3}$ (MeV)"))


# Fiblling histogram with values of p2 and p3
hist_p2.fill#COMPLETE
hist_p3.fill#COMPLETE

# Plotting the histograms
fig, axes = plt.subplots(figsize=(10,5))
plt.subplot(1,2,1)
hist_p2.plot()
plt.subplot(1,2,2)
hist_p3.plot()
plt.show()


## Find the decaying meson! <a name="4."></a>

So far we've learned how to obtain the momentum magnitude but to calculate the energy, we need to know what particles are present in our final state. To find out the exact decays are looking at, we will need to go thought all the possible combinations for a three-body decay involving only pions and kaons, calculate the energy and then we will be ready to reconstruct the invariant mass! 

Since we have only kaons and pions in our three body decay, the possible combinations in the final state we have are:

* $KKK$
* $\pi\pi\pi$
* $\pi KK$
* $\pi K\pi$
* $K\pi\pi$
* $K \pi K$
* $\pi \pi K$
* $KK\pi$

From the PDG, we know that the mass of the kaon and the pion are:

* $m_{\pi} = 139.57$ MeV/c<sup>2</sup>
* $m_{K} = 493.667$ MeV/c<sup>2</sup>

To identify our decay, we want to reconstruct the invariant mass distribution and we expect that this distribution to look like a gaussian function. If we place the wrong final state particle in a given position, for instance, if we say particle 1 is a pion and it was actually supposed to be a kaon, we will see deviations form our ideal gaussian shape. So this will tell us if the combination we are proposing is right or not.


Then let's define the mass of the pion and kaon:

In [ ]:
# Declaring kaon and pion mass in MeV/c^2
mk = 493.667
mpi = 139.57

## Calculating the invariant mass distributions

To test all of our 8 combinations, we will loop through all the events we have in our file and calculate all the eight hypothesis. We will evaluate the energy of the three final state particles, reconstruct the invariant mass and save it into our histogram so we can plot everything together at the end.

### 1. KKK hypothesis

In [ ]:
import uproot
import numpy as np
import matplotlib.pyplot as plt

# Define the histogram properties
hist_bins = 60
hist_range = (1800, 2000)  # Set the appropriate range for your data
masses = []

# Loop through the entries in the DecayTree
for entry in events.iterate(library="np"):
    # Access the variables using their branch names
    p1x = entry["p1_PX"]
    p2x = entry["p2_PX"]
    p3x = entry["p3_PX"]

    p1y = entry["p1_PY"]
    p2y = entry["p2_PY"]
    p3y = entry["p3_PY"]

    p1z = entry["p1_PZ"]
    p2z = entry["p2_PZ"]
    p3z = entry["p3_PZ"]

    # Calculating energy of particles 1, 2, and 3
    e1 = np.sqrt(mk**2 + p1x**2 + p1y**2 + p1z**2)
    e2 = np.sqrt(mk**2 + p2x**2 + p2y**2 + p2z**2)
    e3 = np.sqrt(mk**2 + p3x**2 + p3y**2 + p3z**2)

    # Calculating invariant mass
    mass = np.sqrt((e1 + e2 + e3)**2 - (p1x + p2x + p3x)**2 - (p1y + p2y + p3y)**2 - (p1z + p2z + p3z)**2)
    masses.extend(mass)

# Create the histogram using numpy.histogram
hkkk, edges = np.histogram(masses, bins=hist_bins, range=hist_range)

# Plot the histogram
plt.figure(figsize=(10, 6))
plt.hist(edges[:-1], bins=edges, weights=hkkk, color='blue', alpha=0.7, label='X -> KKK')
plt.xlabel('$m(KKK)$ [MeV/c$^2$]')
plt.ylabel('Entries')
plt.legend()
plt.show()


### 2. $\pi\pi\pi$ hypothesis

In [ ]:
import uproot
import numpy as np
import matplotlib.pyplot as plt

# Define the histogram properties
hist_bins = 60
hist_range = (1500, 2500)  # Set the appropriate range for your data
masses = []

# Loop through the entries in the DecayTree
for entry in events.iterate(library="np"):
    # Access the variables using their branch names
    p1x = entry["p1_PX"]
    p2x = entry["p2_PX"]
    p3x = entry["p3_PX"]

    p1y = entry["p1_PY"]
    p2y = entry["p2_PY"]
    p3y = entry["p3_PY"]

    p1z = entry["p1_PZ"]
    p2z = entry["p2_PZ"]
    p3z = entry["p3_PZ"]

    # Calculating energy of particles 1, 2, and 3
    e1 = np.sqrt(mpi**2 + p1x**2 + p1y**2 + p1z**2)
    e2 = np.sqrt(mpi**2 + p2x**2 + p2y**2 + p2z**2)
    e3 = np.sqrt(mpi**2 + p3x**2 + p3y**2 + p3z**2)

    # Calculating invariant mass
    mass = np.sqrt((e1 + e2 + e3)**2 - (p1x + p2x + p3x)**2 - (p1y + p2y + p3y)**2 - (p1z + p2z + p3z)**2)
    masses.extend(mass)

# Create the histogram using numpy.histogram
hpipipi, edges = np.histogram(masses, bins=hist_bins, range=hist_range)

# Plot the histogram
plt.figure(figsize=(10, 6))
plt.hist(edges[:-1], bins=edges, weights=hpipipi, color='blue', alpha=0.7, label='$X \\to \pi \pi \pi$')
plt.xlabel('$m(\pi \pi \pi)$ [MeV/c$^2$]')
plt.ylabel('Entries')
plt.legend()
plt.show()


# Over to you!

Repeat the same exercise but for the other possible combinations and see if you find any particular one that gives you a peaked structure.

### 3. $\pi KK$ hypothesis

In [ ]:
#COMPLETE

### 4. $\pi K\pi$ hypothesis

In [ ]:
#COMPLETE

### 5. $K\pi\pi$ hypothesis

In [ ]:
#COMPLETE

### 6. $K \pi K$ hypothesis

In [ ]:
#COMPLETE

### 7. $\pi \pi K$ hypothesis


In [ ]:
#COMPLETE

### 8. $KK\pi$ hypothesis

In [ ]:
#COMPLETE

<div class="alert alert-success">
Great Job! By now you have all eight invariant mass plots and our final job is to identify our decay!
</div>

Can you find any of our combinations for which the invariant mass distribution looks like a gaussian function? If yes, which one is that? Did you find one peak? More than one peak?


<details>
    <summary>Answer: </summary>
        KKπ is our decay! We can see two peaks in the distribution, one cetered at around 1870MeV and the other at 1970MeV. These are the D+ and Ds+ mesons! So the processes we are seeing are: D+ -> KKπ and Ds+ -> KK π
</details>


## Extra: Fitting our mass distribution

Now, our last step is to fit this distribution. Now that we have reconstructed the invariant mass, the next step is to fit the histogram. To fit the histogram is to try to find the best function, (or combination of funtions) that reproduce the shape we see in our data. Let's start with a simple example. 

Imagine we have a set of data points and we want to find a line that passes nearby our points.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

#define data
x = np.array([1, 2, 3, 4, 5, 6, 7, 8])
y = np.array([2, 5, 6, 7, 9, 12, 16, 19])

#find line of best fit
a, b = np.polyfit(x, y, 1)

#add points to plot
plt.scatter(x, y)
plt.xlabel("x")
plt.ylabel("y")

#add line of best fit to plot
plt.plot(x, a*x+b, c="red")       

In our plot, the blue dots represent our data and the red line is the best line we can draw that reproduces the behavior of our data. Our LHCb data is not as simple as this example, but the idea is the same! We want to find the best curve that reproduces the shape of the mass distribution we found. It won't be a straight line as our data does not have this kind of pattern, but there are other types of functions that we can use.

### Gaussian function

One very commonly used is the Gaussian function (also known as the normal distribution). We can represent it by the function:

$f(x) = \frac{1}{\sigma \sqrt{2\pi}} e^{-\frac{1}{2} \big(\frac{x-\mu}{\sigma}\big)^2}$

And if we want to draw this function, we can run the following line:

In [ ]:
import numpy as np
import scipy as sp
from scipy import stats
import matplotlib.pyplot as plt

## generate the data and plot it for a gaussian curve

## x-axis for the plot
x_data = np.arange(-5, 5, 0.001)

## y-axis as the gaussian
y_data = stats.norm.pdf(x_data, 0, 1)

## plot data
plt.plot(x_data, y_data)


Interesting... it looks like this gaussian function has some similarities with the mass peak we found! So perhaps we can use this Gaussian function in our fit to the mass peak. When dealing with gaussian functions, two very important features are the mean value of the function, $\mu$, and the width, $\sigma$. The $\mu$ is the value which our curve is cetered at, in this case, the gaussian peak is centered at zero. The $\sigma$ is the width of the distrbution, in this case we can see that the distribution is basically over before -2 and after 2. So the total width here is 1.

<div class="alert alert-warning"> Don't worry if the function has a complicated mathematical expression, the idea here is that since it looks a lot like our peak, we can use this function to learn features of our data!
</div>


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

def gaussian(x, A, mu, sigma):
    return A * np.exp(-1.0 * (x - mu)**2 / (2 * sigma**2))

def fit_function(x, a1, mu1, sigma1, a2, mu2, sigma2):
    return gaussian(x, a1, mu1, sigma1) + gaussian(x, a2, mu2, sigma2)

# Fit the function to the histogram data
bin_centers = 0.5 * (edges[1:] + edges[:-1])  # Calculate bin centers

# Initial guess for the fit parameters
p0 = [max(histogram), 1865, 10, max(histogram), 1905, 10]

fit_params, fit_covariance = curve_fit(fit_function, bin_centers, histogram, p0=p0)
error = np.sqrt(np.diag(fit_covariance))

# Create a smooth curve using the fitted parameters
x_smooth = np.linspace(hist_range[0], hist_range[1], 1000)
y_fit = fit_function(x_smooth, *fit_params)

# Plot the histogram and the fitted curve
plt.figure(figsize=(10, 6))
plt.hist(edges[:-1], bins=edges, weights=histogram, color='blue', alpha=0.7, label='X -> KKpi')
plt.plot(x_smooth, y_fit, 'r-', label='Signal component')
plt.xlabel('$m(KK\pi)$ [MeV/c$^2$]')
plt.ylabel('Entries')
plt.legend()
plt.show()


<div class="alert alert-success">
Great Job! We have now fitted our distribution and identified the peaks! We can verify the mass of the decaying particle by looking at where this peak is centered, which means we have one peak at around 1870MeV (D+) and the other at around 1970MeV (Ds+).
</div>

From the fit, we can confirm the decaying particle by printing the mean values of our Gaussian functions.


In [ ]:
#Printing fitted parameters
# Peak 1

print("The mean values and the uncertainties from the optimization for the first peak:")
print("")

mean_value_1 = "mu = {} +- {}".format(fit_params[1], error[1])
sigma_value_1 = "sigma = {} +- {}".format(fit_params[2], error[2])
print(mean_value_1)
print(sigma_value_1)




In [ ]:
# Peak 2

print("The mean values and the uncertainties from the optimization for the second peak:")
print("")

mean_value_2 = "mu = {} +- {}".format(fit_params[4], error[4])
sigma_value_2 = "sigma = {} +- {}".format(fit_params[5], error[5])
print(mean_value_2)
print(sigma_value_2)

[Return to contents](#c)

---